In [1]:
import pandas as pd
from pathlib import Path

In [2]:
base_path = Path("../external_data")
reads = base_path / "proteomics.xlsx"
assert reads.is_file()

In [3]:
data = pd.read_excel(reads)
data.set_index('Accession', inplace=True)

In [4]:
data.fillna(0., inplace=True)

In [5]:
data.drop('P09874', axis=0, inplace=True) # this is PARP1

In [6]:
mp_cols = [col for col in data.columns if col.startswith('mitoparp')]
parent_cols = [col for col in data.columns if col.startswith("parental")]

In [7]:
cell_line_data = []
for cols in [parent_cols, mp_cols]:
    cell_line_data.append(data[cols].mean(axis=1))
cell_line_data = pd.concat(cell_line_data, axis=1, sort=True)
cell_line_data.columns = ['control', 'mp']

In [8]:
mapping_mc = pd.read_csv(
    base_path / "uniprot_acc_to_ensemble_gene.tsv", sep="\t", index_col=0, header=0
) # this was taken from david.ncifcrf.gov
mapping_mc = mapping_mc[~mapping_mc.index.duplicated(keep='first')]
cell_line_data = cell_line_data[~cell_line_data.index.duplicated(keep='first')]
cell_line_data = pd.concat([cell_line_data, mapping_mc], axis=1, sort=True)

In [9]:
cell_line_data.set_index('To', inplace=True)

In [10]:
cell_line_data.describe()

,control,mp
count,6461.000000,6461.000000
mean,1.025847,0.965629
std,1.502145,1.193348
min,0.000000,0.000000
25%,0.882000,0.849667
50%,1.015667,0.977000
75%,1.130667,1.077333
max,67.047667,66.666667


In [11]:
cell_line_data.to_csv(base_path / "293parp_abundance_ratios_mapped.csv")